In [91]:
import json_lines
import pandas as pd
import pickle
import numpy as np

import lxml.etree as etree # to create gexl file
import datetime

In [ ]:
# unique values in a column
uniq_userid = pd.Series(dtype='float64')
for i in range(1,8):    
    data = pd.read_csv(r"D:\twarc\climatetweetID\localtweets\localtweet_%d.csv" % (i))
    userid = data['user_id_str'].unique()
    userid = pd.Series(userid)
    uniq_userid = uniq_userid.append(userid,ignore_index=True)
    print(i, '\r')  
with open(r"D:\twarc\climatetweetID\sna\uniq_userid.txt", "wb") as fp:   #Pickling
    pickle.dump(uniq_userid, fp)  

In [17]:
# extract uniq user id
with open(r"D:\twarc\climatetweetID\sna\uniq_userid.txt", "rb") as fp:   # Unpickling
    uniq_userid = pickle.load(fp)
uniq_userid = list(map(str,uniq_userid))    

In [19]:
# define function to extract from jsonl
def load_jsonl(file):
    tweets = []
    with open(file, 'rb') as f:
        for tweet in json_lines.reader(f, broken=True):            
            reduced_tweet = {
                'created_at' : tweet['created_at'],
                'id' : tweet['id_str'],
                'username' : tweet['user']['screen_name'],
                'user_joined' : tweet['user']['created_at'][-4:],
                'user_id' : tweet['user']['id_str'],
                'retweet_count' : tweet['retweet_count']                
            }               
            if 'retweeted_status' in tweet:
                reduced_tweet['retweeted_user'] = {
                    'user_id' : tweet['retweeted_status']['user']['id_str'],
                    'username' : tweet['retweeted_status']['user']['screen_name'],
                    'user_joined': tweet['retweeted_status']['user']['created_at'][-4:]
                }                               
            tweets.append(reduced_tweet)
    return (tweets)

In [32]:
# extract from jsonl
for i in range(0,4):
    #tweets = load_jsonl(r"D:\twarc\climatetweetID\sna\tweets_small.jsonl")
    tweets = load_jsonl(r"D:\twarc\climatetweetID\hydratefile\climate_id_0%d.jsonl" % (i))
    tweets = pd.DataFrame(tweets)
    tweets = tweets[tweets['user_id'].isin(uniq_userid)]
    with open(r"D:\twarc\climatetweetID\sna\networktweet\network_tweet_%d.txt" % (i), "wb") as fp:   #Pickling
        pickle.dump(tweets, fp)
    print(i)

0
1
2
3


In [137]:
# read and build network
with open(r"D:\twarc\climatetweetID\sna\networktweet\network_tweet_0.txt", "rb") as fp:   # Unpickling
    tweets = pickle.load(fp)
#tweets = load_jsonl(r"D:\twarc\climatetweetID\sna\networktweet\network_tweet_0.txt")
#tweets = pd.DataFrame(tweets)
#tweets = tweets[tweets['user_id'].isin(uniq_userid)]
tweets = tweets[tweets['retweet_count']> 20000]

tweets_test = tweets.to_dict('records')
tweets_test

[{'created_at': 'Mon Aug 13 10:49:42 +0000 2018',
  'id': '1028956757572374528',
  'username': 'Kingboomboom021',
  'user_joined': '2018',
  'user_id': '1027757751383011328',
  'retweet_count': 70624,
  'retweeted_user': {'user_id': '19725644',
   'username': 'neiltyson',
   'user_joined': '2009'}},
 {'created_at': 'Mon Aug 13 11:31:51 +0000 2018',
  'id': '1028967364593770501',
  'username': 'pixiewolf_',
  'user_joined': '2009',
  'user_id': '27791453',
  'retweet_count': 70624,
  'retweeted_user': {'user_id': '19725644',
   'username': 'neiltyson',
   'user_joined': '2009'}},
 {'created_at': 'Mon Aug 13 12:48:20 +0000 2018',
  'id': '1028986611952840704',
  'username': 'JohnCoo38695683',
  'user_joined': '2018',
  'user_id': '996637186014429184',
  'retweet_count': 94647,
  'retweeted_user': {'user_id': '29275869',
   'username': 'AlbertMacGloan',
   'user_joined': '2009'}},
 {'created_at': 'Mon Aug 13 15:31:34 +0000 2018',
  'id': '1029027688802136064',
  'username': 'SteelPhilFan'

In [138]:
def create_gexf(tweets, filename):
    attr_qname = etree.QName("http://www.w3.org/2001/XMLSchema-instance", "schemaLocation")

    gexf = etree.Element('gexf',
                         {attr_qname : 'http://www.gexf.net/1.3draft  http://www.gexf.net/1.3draft/gexf.xsd'},
                         nsmap={None : 'http://graphml.graphdrawing.org/xmlns/graphml'},
                         version = '1.3')

    graph = etree.SubElement(gexf,
                             'graph',
                             defaultedgetype = 'directed',
                             mode =  'dynamic',
                             timeformat='datetime')
    attributes = etree.SubElement(graph, 'attributes', {'class' : 'node', 'mode' : 'static'})
    #etree.SubElement(attributes, 'attribute', {'id':'country', 'title':'country', 'type' : 'string'})
    #etree.SubElement(attributes, 'attribute', {'id':'region', 'title':'region', 'type' : 'string'})
    etree.SubElement(attributes, 'attribute', {'id':'year', 'title':'year', 'type' : 'integer'})

    nodes = etree.SubElement(graph, 'nodes')
    edges = etree.SubElement(graph, 'edges')

    for tweet in reversed(tweets):
        node = etree.SubElement(nodes,
                                'node',
                                id = tweet['user_id'],
                                Label = tweet['username'],
                                start = datetime.datetime.strptime(tweet['created_at'],'%a %b %d %X %z %Y').isoformat(timespec='seconds'), #Fri Jul 27 07:52:57 +0000 2018
                                end = (datetime.datetime.strptime(tweet['created_at'],'%a %b %d %X %z %Y') + datetime.timedelta(seconds=1)).isoformat(timespec='seconds')
                               )
        attvalues = etree.SubElement(node,'attvalues')
        etree.SubElement(attvalues,
                         'attvalue',
                         {'for' : 'year',
                         'value' : tweet['user_joined']
                         }
                        )
        
        if isinstance(tweet['retweeted_user'], dict):
            etree.SubElement(edges,
                             'edge',
                             {'id' : tweet['id'],
                             'source' : tweet['retweeted_user']['user_id'],
                             'target' : tweet['user_id'],
                             'start' : datetime.datetime.strptime(tweet['created_at'],'%a %b %d %X %z %Y').isoformat(timespec='seconds'), #Fri Jul 27 07:52:57 +0000 2018
                             'end' : (datetime.datetime.strptime(tweet['created_at'],'%a %b %d %X %z %Y') + datetime.timedelta(seconds=1)).isoformat(timespec='seconds')
                             }
                            )
            node = etree.SubElement(nodes,
                                    'node',
                                    id = tweet['retweeted_user']['user_id'],
                                    Label = tweet['retweeted_user']['username'],
                                    start = datetime.datetime.strptime(tweet['created_at'],'%a %b %d %X %z %Y').isoformat(timespec='seconds'), #Fri Jul 27 07:52:57 +0000 2018
                                    end = (datetime.datetime.strptime(tweet['created_at'],'%a %b %d %X %z %Y') + datetime.timedelta(seconds=1)).isoformat(timespec='seconds')
                                   )
            attvalues = etree.SubElement(node,'attvalues')
            etree.SubElement(attvalues,
                             'attvalue',
                             {'for' : 'year',
                             'value' : tweet['retweeted_user']['user_joined']
                             }
                            )


    with open(filename, 'w', encoding='utf-8')as f:
        f.write(etree.tostring(gexf, encoding='utf8', method='xml').decode('utf-8'))
    print('Created gexf.')

In [139]:
create_gexf(tweets_test, 'large_test_network.gexf')

Created gexf.


In [140]:
len(tweets)

155335

In [ ]:
pwd()